# Production API
#### Disclaimer: This is a simplified version of the production API. This notebook is not meant to be ran. It is only meant to be used as a reference.

The following code is utility functions used to get app information or calculate the terms approved for a given application.

In [ ]:
# buying/fxns.py
import json
import os
import re
from textwrap import indent

import requests


def calculate_decision(app):
    """
    Calculate the decision structure for a given application.
    
    Args:
        app (dict): Application dictionary containing terms requested, dealer info and vehicle details
        
    Returns:
        dict: Decision structure containing payment details like cash price, term, APR etc.
                Returns None if required data is missing
    """
    if 'termsRequested' not in app or app['termsRequested'] is None:
        return None
    tr = app['termsRequested']
    if 'term' not in tr:
        tr['term'] = None
    if 'apr' not in tr or tr['apr'] is None or tr['apr'] <= 0:
        tr['apr'] = None
    if 'af' not in tr or tr['af'] is None or tr['af'] <= 0:
        tr['af'] = None
    if 'cashPrice' not in tr:
        tr['cashPrice'] = None
    if 'totalDown' not in tr:
        tr['totalDown'] = 0
    data = {
        'applicationId' : app['applicationId'],
        'dealer': app['dealer'],
        'decisionArray': {
            "decision": "Approval",
            "subDecision": "notifySelect",
            "cashPrice": tr['cashPrice'],
            "term": tr['term'],
            "warranty": 0,
            "gap": 0,
            "tax": 0,
            "titleLicFee": 0,
            "docFee": 0,
            "totalDown": tr['totalDown'],
            "apr": tr['apr'],
            "aquisitionFee": app['dealer']['aquisitionFee'] if 'aquisitionFee' in app['dealer'] else 0,
            "discountFee": 0,
            "amountApproved": tr['af'],
            "notesToOriginationSystem": "",
        },
        'vehicleYear': app['vehicle']['model_year'],
    }
    endpoint = '{}/buying/caculate_decision.php'.format(os.getenv('PRONTO_ENDPOINT_BUYING'))
    rs = requests.post(endpoint, json=data)
    din = json.loads(rs.text)
    if 'data' not in din:
        return None
    data = din['data']
    return {
        'cashPrice': tr['cashPrice'],
        'term': data['term'],
        'totalDown': data['totalDown'],
        'apr': data['apr'],
        'amountApproved': data['amountApproved'],
        'payment': data['payment'],
        'paymentTotal': data['paymentTotal'],
    }

def get_last_decision(app):
    """
    Get the last decision for a given application.
    
    Args:
        app (dict): Application dictionary containing applicationId
        
    Returns:
        dict: Last decision structure containing decision details
                Returns None if required data is missing
    """
    endpoint = '{}/buying/application_last_decision.php'.format(os.getenv('PRONTO_ENDPOINT_BUYING'))
    data = {
        'applicationId': app['applicationId'],
    }
    try:
        rs = requests.post(endpoint, json=data, timeout=30)
        if rs.status_code != 200:
            return None
        din = json.loads(rs.text)
    except requests.exceptions.Timeout:
        return None
    if rs.text is None:
        return None
    return din['last_decision'] if 'last_decision' in din else None


def get_credit_info(app_id: str) -> tuple[any,str]:
    """
    Get the credit information for a given application.
    
    Args:
        app_id (str): Application ID
        
    Returns:
        tuple: Tuple containing credit information and error message
                Returns None if required data is missing
    """
    endpoint = '{}/buying/credit_info.php'.format(os.getenv('PRONTO_ENDPOINT_BUYING'))
    dout = {
        'applicationId' : app_id,
    }
    r = requests.post(endpoint, json=dout)
    if r.status_code != 200:
        return None, 'internal server error'
    din = json.loads(r.text)
    return din['credit_info'] if 'credit_info' in din else None, din['error']

def get_app_milestones(app: any) -> list[any]:
    """
    Get the milestones for a given application.
    
    Args:
        app (dict): Application dictionary containing applicationId
        
    Returns:
        list: List of milestones
                Returns None if required data is missing
    """
    endpoint = '{}/buying/application_milestones.php'.format(os.getenv('PRONTO_ENDPOINT_BUYING'))
    data = {'applicationId': app['applicationId']}
    rs = requests.post(endpoint, json=data)
    din = json.loads(rs.text)
    return [v for _, v in din['milestones'].items()] if 'milestones' in din else []


def get_job_time_months(contact: any) -> any:
    """
    Get the job time in months for a given contact.
    
    Args:
        contact (dict): Contact dictionary containing employers
        
    Returns:
        any: Job time in months
                Returns None if required data is missing
    """
    job_time_months = None
    if len(contact['employers']) > 0:
        e = contact['employers'][0]
        if 'timeAtYears' in e:
            e['timeAtYears'] = str(e['timeAtYears']).replace('+', '')
            if e['timeAtYears'] != '':
                if job_time_months is None:
                    job_time_months = 0
                tay = 0
                try:
                    tay = int(e['timeAtYears'])
                except:
                    tay = 0
                    pass
                job_time_months += (tay * 12)    
            
        if 'timeAtMonths' in e:
            if e['timeAtMonths'] != '':
                try:
                    tam = int(e['timeAtMonths'])
                except:
                    pass
                else:
                    if job_time_months is None:
                        job_time_months = 0
                    job_time_months += tam
    return job_time_months

def get_monthly_income(contact:any) -> any:
    """
    Get the monthly income for a given contact.
    
    Args:
        contact (dict): Contact dictionary containing employers
        
    Returns:
        any: Monthly income
                Returns None if required data is missing
    """
    monthly_income = None
    if len(contact['employers']) > 0:
        e = contact['employers'][0]
        pay_interval = 'Monthly'
        if 'payInterval' in e:
            pay_interval = e['payInterval']

            if 'grossIncome' in e and e['grossIncome'] is not None:
                if monthly_income is None:
                    monthly_income = 0
                if pay_interval == 'Monthly':
                    monthly_income += float(e['grossIncome'])
                elif pay_interval == 'Weekly':
                    monthly_income += (float(e['grossIncome']) * 4)
                elif pay_interval == 'BiWeekly':
                    monthly_income += (float(e['grossIncome']) * 2)
                elif pay_interval == 'Annual':
                    monthly_income += (float(e['grossIncome']) / 12)
    return monthly_income

def get_res_time_months(contact: any) -> any:
    """
    Get the residence time in months for a given contact.
    
    Args:
        contact (dict): Contact dictionary containing addresses
        
    Returns:
        any: Residence time in months
                Returns None if required data is missing
    """
    res_time_months = None
    if len(contact['addresses']) > 0:
        a = contact['addresses'][0]
        if 'type' in a and a['type'] == 'Previous':
            return res_time_months
        if 'timeAtYears' in a:
            a['timeAtYears'] = str(a['timeAtYears']).replace('+', '')
            if a['timeAtYears'] != '':
                if res_time_months is None:
                    res_time_months = 0
                tay = 0
                try:
                    tay = int(a['timeAtYears'])
                except:
                    tay = 0
                    pass
                res_time_months += (tay * 12)
        if 'timeAtMonths' in a:
            if a['timeAtMonths'] != '':
                try:
                    tam = int(e['timeAtMonths'])
                except:
                    pass
                else:
                    if res_time_months is None:
                        res_time_months = 0
                    res_time_months += tam
    return res_time_months

def set_precision(v):
    """
    Set the precision for a given value.
    
    Args:
        v (float): Value to be rounded
        
    Returns:
        float: Rounded value
                Returns None if required data is missing
    """
    if '.' in str(v):
        return round(v, 2)
    return v


# From App to Model Features
The following is an example of how one might transform application data for model processing.<br>
It entails flattening the application data and keeping it in the same order in which it was used to train the model.

In [ ]:
# buying/service.py
import time

from injector import inject

from db import DB

from buying.fxns import (
    calculate_decision,
    get_credit_info,
    get_job_time_months,
    get_last_decision,
    get_monthly_income,
    get_res_time_months,
    set_precision,
)


class AppService:
    """
    Service class for handling application data and operations.
    """
    @inject
    def __init__(self, db: DB):
        self.db = db
    # ./__init__

    def format_app_for_model(self, app_id: str):
        """
        Format application data for model processing.
        
        Args:
            app_id (str): Application ID to retrieve and format
            
        Returns:
            dict: Formatted application data
            
        Raises:
            Exception: If application meta or application not found, or if closed application
                    has invalid decision status
        """
        col_meta = self.db.database('Buying')['applicationMeta']
        col_app = self.db.database('Buying')['application']
        meta = col_meta.find_one({'applicationId': app_id})
        if meta is None:
            raise Exception('Application meta not found')
        
        proj = {
            '_id' : 1, 
            'applicationId' : 1, 
            'termsRequested' : 1, 
            'termsApproved' : 1, 
            'termsDecisioned': 1,
            'contact' : 1, 
            'dealer': 1, 
            'vehicle': 1, 
            'status': 1,
            'submittedby': 1,
        }

        app_statuses = [
            # denied statuses
            'Denial', 

            # pending statuses
            'Pending Approval', 

            # approved statuses
            'Approved', 
            'Accepted', 
            'Funded', 
            'Counter',

            # closed statuses
            'Closed'
        ]

        try:
            find = {'applicationId' : meta['applicationId']}
            app = col_app.find_one(find, projection=proj)
            if app is None:
                raise Exception('Application not found')
                
            if meta['status'] == 'Closed':
                td = app['termsDecisioned']
                dapp = get_last_decision(app)
                if dapp is None:
                    raise Exception('Closed: No decision found')
                    
                if dapp['status'] not in app_statuses:
                    raise Exception('Closed: decision not in statuses')
                    
                app = dapp
            tr = app['termsRequested']
            if 'termsApproved' not in app or app['termsApproved'] is None:
                ta = calculate_decision(app)
                if ta is None:
                    raise Exception('Termed Approved invalid')                    
            else:
                ta = app['termsApproved']
            # structure info
            cash_down = float(tr['totalDown']) if 'totalDown' in tr else 0
            if ta['amountApproved'] is None:
                raise Exception('Termed Approved invalid')
                
            amount_approved = float(ta['amountApproved'])
            try:
                est_payment = float(ta['paymentTotal'])
            except:
                est_payment = None

            # employment info
            buyer = app['contact'][0]
            job_time_months = get_job_time_months(buyer)
            monthly_income = get_monthly_income(buyer)
            res_time_months = get_res_time_months(buyer)

            credit_chargeoffs, credit_child_supports, credit_wage_earners = 0, 0, 0
            credit_bankruptcies, credit_repos, credit_foreclosures, = 0, 0, 0
            credit_past_due_bankruptcy, credit_past_due_medical, credit_past_due_auto, credit_past_due_mortgage = -1, -1, -1, -1
            credit_collections = 0

            # credit info
            no_credit = False
            credit_chargeoffs = -1
            credit_child_supports = -1
            credit_wage_earners = -1
            credit_bankruptcies = -1
            credit_repos = -1
            credit_foreclosures = -1
            credit_collections = -1
            credit_past_due = -1
            credit_current_balance = -1
            credit_monthly_payment = -1
            credit_months_since_repo = -1
            credit_months_since_bankruptcy = -1
            credit_months_since_foreclosure = -1
            credit_months_since_chargeoff = -1
            credit_age_months = -1
            credit_dpd_30 = -1
            credit_dpd_60 = -1
            credit_dpd_90 = -1
            credit_dpd_120 = -1
            credit_dpd_30_last_12_percent = -1
            credit_dpd_60_last_12_percent = -1
            credit_dpd_90_last_12_percent = -1
            credit_dpd_120_last_12_percent = -1
            credit_dpd_30_last_24_percent = -1
            credit_dpd_60_last_24_percent = -1
            credit_dpd_90_last_24_percent = -1
            credit_dpd_120_last_24_percent = -1
            credit_line_suspended = False
            vantage_score = -1
            credit_info, err = get_credit_info(app['applicationId'])
            if err != '' or credit_info is None:
                no_credit = True
            if not no_credit:
                analysis = credit_info['credit_analysis']
                if analysis is None or len(analysis) == 0:
                    no_credit = True

                    if len(analysis['trades']) == 0 and len(analysis['collections']) == 0 and len(analysis['public_records']) == 0:
                        no_credit = True
                    credit_chargeoffs = len(analysis['charge_off_line_items']) if analysis['charge_off_line_items'] is not None else 0
                    credit_child_supports = len(analysis['child_support_line_items']) if analysis['child_support_line_items'] is not None else 0
                    credit_wage_earners = len(analysis['wage_earner_line_items']) if analysis['wage_earner_line_items'] is not None else 0
                    if analysis['bankruptcy']['line_items'] is not None:
                        credit_bankruptcies = len(analysis['bankruptcy']['line_items'])
                        for b in analysis['bankruptcy']['line_items']:
                            if 'past_due' in b:
                                credit_past_due_bankruptcy += b['past_due']
                    credit_repos = len(analysis['repo']['line_items']) if analysis['repo']['line_items'] is not None else 0
                    credit_foreclosures = len(analysis['foreclosure']['line_items']) if analysis['foreclosure']['line_items'] is not None else 0
                    credit_collections = len(analysis['collections'])
                    credit_line_suspended = analysis['credit_line_suspended']
                    credit_past_due = analysis['credit_summary']['totalAmount']['pastDue']
                    credit_current_balance = analysis['credit_summary']['totalAmount']['currentBalance']
                    credit_monthly_payment = analysis['credit_summary']['totalAmount']['monthlyPayment']
                    if 'mortgageAmount' in analysis['credit_summary']:
                        credit_past_due_mortgage = analysis['credit_summary']['mortgageAmount']['pastDue']
                    cat_map = analysis['trade_totals_category_map']
                    if 'auto_loan' in cat_map:
                        credit_past_due_auto = cat_map['auto_loan']['past_due']
                    if 'medical' in cat_map:
                        credit_past_due_medical = cat_map['medical']['past_due']
                    credit_months_since_repo = analysis['repo']['months_since']
                    credit_months_since_bankruptcy = analysis['bankruptcy']['months_since']
                    credit_months_since_foreclosure = analysis['foreclosure']['months_since']
                    credit_months_since_chargeoff = analysis['charge_off_months_since']
                    infile_date = time.strptime(analysis['infile_date'], "%Y-%m-%d") if analysis['infile_date'] != '' else None
                    if infile_date is not None:
                        credit_age_months = (time.localtime().tm_year - infile_date.tm_year) * 12 + time.localtime().tm_mon - infile_date.tm_mon
                    credit_dpd_30 = analysis['dpd30']
                    credit_dpd_60 = analysis['dpd60']
                    credit_dpd_90 = analysis['dpd90']
                    credit_dpd_120 = analysis['dpd120']

                    # the following data is used to see if more recent credit vs. older credit is better
                    tm_map = analysis['trade_totals_time_map']
                    if tm_map is not None:
                        m12 = tm_map['12_months'] if '12_months' in tm_map else None
                        m24 = tm_map['24_months'] if '24_months' in tm_map else None
                        if m12 is not None:
                            payment_pattern = m12['payment_pattern_counts']
                            credit_dpd_30_last_12_percent = payment_pattern['2'] / m12['total_payments'] if m12['total_payments'] > 0 else 0
                            credit_dpd_60_last_12_percent = payment_pattern['3'] / m12['total_payments'] if m12['total_payments'] > 0 else 0
                            credit_dpd_90_last_12_percent = payment_pattern['4'] / m12['total_payments'] if m12['total_payments'] > 0 else 0
                            credit_dpd_120_last_12_percent = payment_pattern['5'] / m12['total_payments'] if m12['total_payments'] > 0 else 0
                        if m24 is not None:
                            payment_pattern = m24['payment_pattern_counts']
                            credit_dpd_30_last_24_percent = payment_pattern['2'] / m24['total_payments'] if m24['total_payments'] > 0 else 0
                            credit_dpd_60_last_24_percent = payment_pattern['3'] / m24['total_payments'] if m24['total_payments'] > 0 else 0
                            credit_dpd_90_last_24_percent = payment_pattern['4'] / m24['total_payments'] if m24['total_payments'] > 0 else 0
                            credit_dpd_120_last_24_percent = payment_pattern['5'] / m24['total_payments'] if m24['total_payments'] > 0 else 0

                    vantage_score = analysis['vantage_score'].replace("+", "")
                    vantage_score = int(vantage_score) if vantage_score != '' else None

            # vehicle info
            vehicle = app['vehicle']
            ltv_wholesale_clean = None

            if 'value' in vehicle and 'data' in vehicle['value'] and 'wholesale' in vehicle['value']['data']:
                try:
                    clean_total = float(vehicle['value']['data']['wholesale']['clean']['total'])
                    ltv_wholesale_clean = (amount_approved / clean_total) * 100 if clean_total > 0 else None
                except(e):
                    print(e)
                    raise Exception('Vehicle value error')
            return {
                "cash_down": cash_down,
                "amount_approved": amount_approved,
                "monthly_income": set_precision(monthly_income),
                "job_time_months": job_time_months,
                "res_time_months": res_time_months,
                "ltv_wholesale_clean": set_precision(ltv_wholesale_clean),
                "credit_chargeoffs": credit_chargeoffs,
                "credit_child_supports": credit_child_supports,
                "credit_wage_earners": credit_wage_earners,
                "credit_bankruptcies": credit_bankruptcies,
                "credit_repos": credit_repos,
                "credit_foreclosures": credit_foreclosures,
                "credit_collections": credit_collections,
                "credit_line_suspended": credit_line_suspended,
                "credit_past_due": credit_past_due,
                "credit_past_due_bankruptcy": credit_past_due_bankruptcy,
                "credit_past_due_medical": credit_past_due_medical,
                "credit_past_due_auto": credit_past_due_auto,
                "credit_past_due_mortgage": credit_past_due_mortgage,
                "credit_current_balance": credit_current_balance,
                "credit_monthly_payment": credit_monthly_payment,
                "credit_months_since_repo": credit_months_since_repo,
                "credit_months_since_bankruptcy": credit_months_since_bankruptcy,
                "credit_months_since_foreclosure": credit_months_since_foreclosure,
                "credit_months_since_chargeoff": credit_months_since_chargeoff,
                "credit_age_months": credit_age_months,
                "credit_dpd_30": credit_dpd_30,
                "credit_dpd_60": credit_dpd_60,
                "credit_dpd_90": credit_dpd_90,
                "credit_dpd_120": credit_dpd_120,
                "credit_dpd_30_last_12_percent": credit_dpd_30_last_12_percent,
                "credit_dpd_60_last_12_percent": credit_dpd_60_last_12_percent,
                "credit_dpd_90_last_12_percent": credit_dpd_90_last_12_percent,
                "credit_dpd_120_last_12_percent": credit_dpd_120_last_12_percent,
                "credit_dpd_30_last_24_percent": credit_dpd_30_last_24_percent,
                "credit_dpd_60_last_24_percent": credit_dpd_60_last_24_percent,
                "credit_dpd_90_last_24_percent": credit_dpd_90_last_24_percent,
                "credit_dpd_120_last_24_percent": credit_dpd_120_last_24_percent,
                "no_credit": 1 if no_credit else 0,
            }
        except Exception as e:
            raise e


# Flask Server
The code below is a flask server that uses the AppService class to classify the decision for a given application.<br>
It uses the format_app_for_model method to transform application data for model processing.<br>
It then checks for missing fields and loads the model to predict the decision.<br>
It returns the decision and error message.

In [ ]:
# server.py
import json
import logging
import os

import joblib
import pandas as pd
from flask import Flask, request, jsonify


from injector import inject

from buying.service import AppService

# configurePrintService()
app = Flask(__name__)

@inject
@app.route("/buying/model/classify/decision", methods=['POST'])
def buying_model_classify_decision(s: AppService):
    """
    Classify the decision for a given application.
    
    Args:
        s (AppService): AppService instance
        
    Returns:
        tuple: Tuple containing the decision and error message
                Returns None if required data is missing
    """
    rq = json.loads(request.data)
    app_id = rq['applicationId']
    ret = {'error': '', 'decision': ''}
    if app_id == '':
        ret['error'] = 'invalid request'
        return jsonify(ret), 400
    try:
        formatted = s.format_app_for_model(app_id)
        if formatted is None:
            ret['error'] = 'application not found'
            return jsonify(ret), 500
        
        # check for missing fields
        missing_fields = []
        for k, v in formatted.items():
            if v == None:
                missing_fields.append(k)
        if len(missing_fields) > 0:
            ret['error'] = 'missing fields: {}'.format(missing_fields)
            return ret
        
        df = pd.DataFrame(formatted, index=[0])
        df = df.drop(columns=['no_credit', 'cash_down', 'amount_approved'])

        model = 'neural_network'
        pipeline = joblib.load('buying_classify_decision_{}_credit.pkl'.format(model))
        decision_num = pipeline.predict(df)[0]
        prob = pipeline.predict_proba(df)[0]
        # 1 = Approved, 0 = Denial
        decision = 'Approved' if decision_num == 1 else 'Denial'
        ret['predicted_decision'] = decision
        ret['confidence'] = prob[decision_num]
        ret['prob_decline'] = prob[0]
        ret['prob_approve'] = prob[1]
        ret['credit'] = True
        ret['model'] = model
        ret['model_features'] = formatted
        return jsonify(ret), 200
    except Exception as e:
        print(e)
        ret['error'] = 'an error occurred'
        return jsonify(ret), 500

if __name__ == '__main__':
    # NOTE: 
    if os.getenv('PRONTO_ENV') == os.getenv("PRONTO_ENV_TYPE_DEV") or os.getenv('PRONTO_ENV') == os.getenv("PRONTO_ENV_TYPE_LOCAL"):
        print("server listening in debug mode on :{}".format(os.getenv('PY_PORT')))
        app.logger.setLevel(logging.INFO)
        app.run(debug=True, host="0.0.0.0", port=os.getenv('PY_PORT'))
    